# CTW dataset tutorial (Part 2: Classification baseline)

In this part of the turotial, we will show you:

  - [Framework of classification baseline](#Framework-of-classification-baseline)
  - [Training steps](#Training-steps)
  - [Predicting steps](#Training-steps)
  - [Results format and evaluation API](#Results-format-and-evaluation-API)
  - [Evaluate results locally](#Evaluate-results-locally)

Notes:

  > This notebook MUST be run under `$CTW_ROOT/examples`.
  >
  > All our code SHOULD run on `Linux>=3` with `Python>=3.4`. We make it compatible with `Python>=2.7` with best effort.

## Framework of classification baseline

We regard the text recoginition problem as a classification problem. We train models using [Tensorflow](https://www.tensorflow.org/).

We only consider recognition of the top 1000 frequent observed character categories. We give up to recognize other categories, which must will leads a failure on those categories.

The _magic number_ `1000` is written in `classification/settings.py`. You may modify it if you want to train with another number of categories.

For each of the character instances, we take following operations.

  1. crop the image region around it
  2. (training step only) randomly adjust saturation, brightness, contrast
  3. (training step only) randomly apply an affine transform
  4. per instance standardization 
  5. resize to fit the input of classification models
  6. feed to each of the classification models


## Training steps

Notes:

  > Before you run any scripts, please ensure you have requirements installed, which are described in `Tutorial part 1: Basics`.
  >
  > We train models on a desktop with 32 GB RAM. If your RAM is less than 32 GB, some code may fail.
  >
  > We train models on NVIDIA GTX TITAN X, which GPU memory is 12 GB. If your GPU memory is less than 12 GB, you may need to turn down `batch_size` in `cfgs` in `classification/train.py`

#### Decide categories
Decide which categories are the top 1000 frequent observed character categories, and save to `products/cates.json`.

In [ ]:
!cd ../classification && python3 decide_cates.py

#### Create pickles

Crop the image region around character instances, and save pickles to `products/*.pkl`, in order to avoid frequently reading `.jpg` files.

Notes:

  > Due to `pickle` module is not fully compatible between Python 2 and Python 3, we restrict each of all python scripts using pickle to use Python 3 with an assertion `assert six.PY3`, including following sections and other tutorials. You MAY change all of them to `assert six.PY2` if needed, but you SHOULD NOT perform a mixture use of Python 2 pickle and Python 3 pickle.

In [ ]:
!cd ../classification && python3 create_pkl.py

#### Run train scripts

Run train script with command line argument `alexnet_v2` to train `AlexNet v2`. Other choices are `overfeat`, `inception_v4`, `resnet_v2_50` and `res_net_v2_152`.

Train logs and checkpoints are saved to `classification/products/train_logs_alexnet_v2/`. You can run `tensorboard` to see detailed logs.

This script generates a mount of logs and takes a long time, so we recommand you to run it with `/bin/bash` instead of running it directly in jupyter notebook.

Time cost estimation (NVIDIA GTX TITAN X):

  - **alexnet_v2**: 0.2 sec / step, 6 hours in total
  - **resnet_v2_152**: 1.2 sec / step, 33 hours in total
  - **others**: 1.0 sec / step, 28 hours in total

Notes:

  > If training step become slower and slower (e.g. > 2 sec / step), you can just press Ctrl+C to interrupt it, execute `sudo sh -c "echo 3 >/proc/sys/vm/drop_caches"` to drop caches, and rerun `train.py`. It will automatically resume from the latest checkpoint. We save checkpoints per 1200 seconds, and this can be modified by `save_interval_secs` in `cfg_common` in `classification/train.py`. Do not run it along with other memory intensive applications.
  >
  > If you get a `CUDNN_STATUS_BAD_PARAM` error, you may turn down `per_process_gpu_memory_fraction` in `classification/train.py`.
  >
  > When training `resnet_v2_152`, tensorflow run training step and summary step at the same time may run out of memory (OOM). You may set `save_summaries_secs` in `cfgs` in `classification/train.py` to infinity (e.g. 999999) to disable summary step.
  >
  > You can modify `cfgs` in `classification/train.py` to add or delete models. All avaliable models are described in `classification/slim/nets/nets_factory.py`, but we have not tested whether each of the models is suitable to our dataset.
  >
  > You can update TensorFlow-Slim from [source](https://github.com/tensorflow/models/tree/master/research/slim), but please keep our customized modified `slim/train_image_classifier.py` and `slim/eval_image_classifier.py`.

In [ ]:
!cd ../classification && python3 train.py alexnet_v2

In [ ]:
# during training, you can browse train logs using tensorboard
!tensorboard --logdir=../classification/products/train_logs_alexnet_v2/

#### Download trained models

Since training takes a lot of energy and we hate global warming, we provide the `checkpoint` trained as described above.

In [ ]:
# TODO: download trained models

## Predicting steps

Just like [training steps](#Training-steps), only need to substitute `train.py` with `eval.py`.

This step will feed each of the character instances in classification testing set to the model, and save the output end point (so called _logits_) of the model to `classification/products/eval_alexnet_v2.pkl`.

Then, for each of the character instances, we sort the `logits`, and output the Top-5 candicates for each of character instances to `classification/products/predictions_alexnet_v2.jsonl`.

In [ ]:
!cd ../classification && python3 eval.py alexnet_v2

## Results format and evaluation API

Classification results MUST be UTF-8 encoded [JSON Lines](http://jsonlines.org/), each line MUST match the corresponding line in `Classification testing set annotations`, which is described in `Tutorial part 1: Basics`.

```
result (corresponding to one line in .jsonl):
{
    predictions: [prediction_0, prediction_1, prediction_2, ...],
}

prediction:
[candidate_0, candidate_1, candidate_2, ...]      # there MUST be at least 5 candidates

candidate: str                                    # length is usually 1, otherwise this must be a failed prediction
```

Our evaluation API in `pythonapi/eval_tools.py` works as follows.

  1. Check prediction (PR) file has the same number of lines as the grount truth (GT) file. Otherwise, return error.
  2. Check each line of the PR file is valid JSON, and conform to the results format. Otherwise, return error.
  3. Check PR provide the same number of predictions as the number of Chinese character instances in GT. Otherwise, return error.
  4. Count the number of instances and the number of matched instances for each of attributes combination and each of sizes, respectively.

If no error, the data struct for the output of evaluation API is described below.

```
output:
{
    error: 0,
    performance: {
      all: size_performance,
      large: size_performance,
      medium: size_performance,
      small: size_performance,
    }
}

size_performance:
[attr_performance_0, attr_performance_1, attr_performance_2, ..., attr_performance_63]

attr_performance:
{
    n: int,
    recalls: {
      1: int,
      5: int,
    }
}  
```

`k` in `attr_performance_k` is represented in bitwise. e.g. `k = 5` (`000101` in binary) means:

| Attribute | Yes or no |
| --------- | --------- |
| occluded  | 1 |
| bgcomplex | 0 |
| distorted | 1 |
| raised    | 0 |
| wordart   | 0 |
| handwritten | 0 |

corresponding to character instances with attributes combination `occluded & ~bgcomplex & distorted & ~raised & ~wordart & ~handwritten`.

Notes:

  > Since our evaluation API computes both top-1 accuracy and top-5 accuracy, you MUST provide at least 5 candidates for each of the instances.

## Evaluate results locally

If you don't have ground truth of testing set, you cannot run following scripts.

#### Gather statistics

Run this script to gather statistics. This step will generate:

  - `judge/products/stat_frequency.json`: the frequency in the whole dataset, both training set and testing set
  - `judge/products/plots/stat_attributes.pdf`: (discribed in our paper)
  - `judge/products/plots/stat_instance_size.pdf`: (discribed in our paper)
  - `judge/products/plots/stat_most_freq.pdf`: (discribed in our paper)
  - `judge/products/plots/stat_num_char.pdf`: (discribed in our paper)
  - `judge/products/plots/stat_num_uniq_char.pdf`: (discribed in our paper)

Notes:

> This step requires network connection to download `SimHei.ttf`, i.e. a Chinese font file. This font file is used in rendering chinese in our statistics, and some other results.

In [ ]:
!cd ../judge && python3 statistics_in_paper.py

#### Evaluate classification performance

In this step, we will output:

  - `<stdout>`: classification performance with each of the attributes
  - `<stdout>`: classification performance for top-10 most frequent character categories
  - `judge/products/plots/cls_precision_by_attr_size_(model_name).pdf`: (discribed in our paper)
  - `judge/products/plots/cls_precision_by_model_size.pdf`: performance for each of models and each of sizes
  - `judge/products/explore_cls.html`: performance for each of models and each of sizes

If you run `classification_perf.py` without command line arguments, it will evaluate all models listed in `cfgs` in `predictions2html.py`.

Notes:

  > You may result in a higher performance than paper. The reason may be our models are trained on TRAIN+VAL, but you are using validation set as testing set now.
  >
  > This step requires network connection to download `Chart.min.js` used to generate `explore_cls.html`.

In [ ]:
!cd ../judge && python3 classification_perf.py alexnet_v2

#### Show results of character instances

Besides `predictions`, our baseline also write `probabilities` to results to enable showing confidence probabilities for each of the predictions.

This step will generate:

  - `judge/products/test_cls_cropped.pkl`: a cache file to avoid frequently reading .jpg files
  - `judge/products/predictions_compare.html`: (discribed in our paper)

If you run `predictions2html.py` without command line arguments, it will evaluate all models listed in `cfgs`.

Then, you can browse `judge/products/predictions_compare.html`.

In [ ]:
!cd ../judge && python3 predictions2html.py alexnet_v2